In [17]:

import os
import csv
import glob
from PIL import Image
from typing import Tuple, Any
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt
import torch.optim as optim
import random


In [6]:
class CustomImageDataset(Dataset):
  def __init__(self, path_to_annotation_file: str, transform: Any=None, target_transform: Any=None) -> None:
    self.path_to_annotation_file = path_to_annotation_file
    self.dataset_info = pd.read_csv(path_to_annotation_file, sep=';', header=0)
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self) -> int:
    return len(self.dataset_info)

  def __getitem__(self, index: int) -> Tuple[torch.tensor, int]:
    path_to_image = self.dataset_info.iloc[index, 0]
    image = cv2.cvtColor(cv2.imread(path_to_image), cv2.COLOR_BGR2RGB)
    label = self.dataset_info.iloc[index, 1]

    if self.transform:
      image = self.transform(image)
    if self.target_transform:
      label = self.target_Transform(label)
      
    return image, label

In [36]:
list_abs_path=[]
list_class_mark=[]
with open("random_annotation.csv", encoding='utf-8') as file:
    reader = csv.reader(file, delimiter = ";")
    for row in reader:
        list_abs_path.append(row[0])
 

list_abs_path.pop(0)   
random.shuffle(list_abs_path)
   
with open("random_annotation.csv", encoding='utf-8') as file:
    reader = csv.reader(file, delimiter = ";")
    for i in range(int(len(list_abs_path))):
        for row in reader:
            if row[0] == list_abs_path[i]:
                list_class_mark.append(row[2])


print(list_abs_path[5])  
print(list_class_mark[5])            

C:\Users\dstep\OneDrive\Рабочий стол\lab3 py\lab3_py-1\dataset\rose\0793.jpg


IndexError: list index out of range

In [ ]:
   
with open("train_anntotation.csv", mode="w", encoding='utf-8') as w_file:
        file_writer = csv.writer(w_file, delimiter=";", lineterminator="\r")
        for i in range(1, int(len(list_abs_path)*0.8)):
            file_writer.writerow([list_abs_path[i], list_class_mark[i]])
            
with open("test_anntotation.csv", mode="w", encoding='utf-8') as w_file:
        file_writer = csv.writer(w_file, delimiter=";", lineterminator="\r")
        for i in range(int(len(list_abs_path)*0.8), int(len(list_abs_path)*0.9)):
            file_writer.writerow([list_abs_path[i], list_class_mark[i]])
            
with open("val_anntotation.csv", mode="w", encoding='utf-8') as w_file:
        file_writer = csv.writer(w_file, delimiter=";", lineterminator="\r")
        for i in range(int(len(list_abs_path)*0.9), len(list_abs_path)):
            file_writer.writerow([list_abs_path[i], list_class_mark[i]])
            


In [9]:
rand_list = np.random.randint(1, len(list_abs_path),size=10)

fig = plt.figure()
i=1
for rand in rand_list:
    fig.add_subplot(2,5,i)
    image = Image.open(list_abs_path[rand])
    plt.imshow(image)
    i+=1

plt.show()

ValueError: low >= high

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

custom_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Resize(
                                                        (224, 224)),
                                                    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = CustomImageDataset('train_annotation.csv', custom_transforms)

cuda:0


In [ ]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super(CNN, self).__init__()

        self.conv_1 = nn.Conv2d(3, 16, kernel_size=3, padding=0, stride=2)
        self.conv_2 = nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2)
        self.conv_3 = nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.max_pool = nn.MaxPool2d(2)

        self.fc_1 = nn.Linear(576, 10)
        self.fc_2 = nn.Linear(10, 1)

    def forward(self, x: torch.tensor) -> torch.tensor:
        output = self.relu(self.conv_1(x))
        output = self.max_pool(output)
        output = self.relu(self.conv_2(output))
        output = self.max_pool(output)
        output = self.relu(self.conv_3(output))
        output = self.max_pool(output)

        output = nn.Flatten()(output)
        output = self.relu(self.fc_1(output))
        output = nn.Sigmoid()(self.fc_2(output))
        return output